In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'ATU1UYGD2FQ4OAFPPKKKHZLEM3U1EVU1EN0EMF4JWVEE2CXS' # your Foursquare ID
CLIENT_SECRET = 'ZGKMZZU1CEBO2NIDXJKASIU1PPSGWTBIIZ1JJLS51LJLCUFO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: ATU1UYGD2FQ4OAFPPKKKHZLEM3U1EVU1EN0EMF4JWVEE2CXS


In [26]:
# type your answer here
LIMIT = 200 # Maximum is 100
cities = ["New York, NY", "Los Angeles, CA"]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d146941735") 
    results[city] = requests.get(url).json()

In [28]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [16]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of deli's in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of deli's in New York, NY =  252
Showing Top 100
Total number of deli's in Los Angeles, CA =  225
Showing Top 100


In [17]:
maps[cities[0]]

In [18]:
maps[cities[1]]

# So what we see here is that deli's in Los Angeles, CA are a further apart of each other which is not that good for tourists which have to travel further and further distances to get what they want, however to make sure of this statement We can take the mean distances of all of the deli's and compare it to the coordinates. How you ask?

So what will happen is that I will get the mean location of deli's in New York and Los Angeles and see how far away they are from each other.

In [19]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.027536518693738632
Los Angeles, CA
Mean Distance from Mean coordinates
0.1271542277134195


In [20]:
maps[cities[0]]

In [21]:
maps[cities[1]]

# What we see here is that the Mean Distance from Mean coordinates is 0.027536518693738632 which is approximately a sixth of what Los Angeles is which is, 0.1271542277134195

# What we get from this is that even though Los Angeles has good bacon, egg, and cheese sandwiches it isn't as convenient for tourists as if they went to New York
